# Fine-Tune & Serve Llama3 with Kubeflow PytorchJob in a Kubeflow Pipeline

This Notebook will do the following:
1. Fine-tune meta-llama/Llama-3.1-8B-Instruct model on KubeCon, India 2024 dataset using distributed training with [Kubeflow PytorchJob](https://www.kubeflow.org/docs/components/training/overview/).
2. Serve the fine-tuned model using Kserve.
 
We are using [Kubeflow Pipelines](https://www.kubeflow.org/docs/components/pipelines/) to run this end-to-end LLM pipeline.


Llama3 model: https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct

KubeCon, India 2024 dataset: https://huggingface.co/datasets/aishwaryayyy/events_data

This Notebook requires:
1. 1 GPU on your Kubernetes cluster for fine-tuning and later serving the fine-tuned model
2. 1 GPU on your Notebook node to load the fine-tuned model by merging PEFT weights.

We need to install Kubeflow Pipeline packages and import the dependencies.

In [ ]:
pip install kfp kfp-kubernetes

In [ ]:
from typing import List
from kfp import client
from kfp import dsl
from kfp.dsl import Dataset
from kfp.dsl import Input
from kfp.dsl import Model
from kfp.dsl import Output

## Fine-Tune Llama3 model with KubeCon dataset

In this component, use TrainingClient() to create PyTorchJob which will fine-tune Llama3 model on 1 worker with 1 GPU.

Specify the required packages in the *dsl.component* decorator. We would need kubeflow-pytorchjob, kubeflow-training[huggingface] and numpy packages in this Kubeflow component.

Replace the HUGGINGFACE_TOKEN with your own token. It should have access to [Llama3 model](https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct).

In [ ]:
@dsl.component(packages_to_install=['kubeflow-pytorchjob', 'kubeflow-training[huggingface]','numpy<1.24'])
def finetune_model():

    from kubeflow.training.api.training_client import TrainingClient
    from kubeflow.storage_initializer.s3 import S3DatasetParams
    from kubeflow.storage_initializer.hugging_face import (
       HuggingFaceModelParams,
       HuggingFaceTrainerParams,
       HuggingFaceDatasetParams,
    )
    from kubeflow.storage_initializer.constants import INIT_CONTAINER_MOUNT_PATH
    from peft import LoraConfig
    import transformers
    from transformers import TrainingArguments
    from kubeflow.training import constants
    
    # create a training client, pass config_file parameter if you want to use kubeconfig other than "~/.kube/config"
    client = TrainingClient()
    OUTPUT = INIT_CONTAINER_MOUNT_PATH + "/output/llama-3.1-8B-kubecon"
    HUGGINGFACE_TOKEN = "YOUR_HUGGINGFACE_TOKEN"
    
    # mention the model, datasets and training parameters
    client.train(
       name="llama-3-1-8b-kubecon",
       num_workers=1,
       num_procs_per_worker=1,
       # specify the storage class if you don't want to use the default one for the storage-initializer PVC
       storage_config={
           "size": "100Gi",
           "storage_class": "nfs-storage",
       },
       model_provider_parameters=HuggingFaceModelParams(
           model_uri="hf://meta-llama/Llama-3.1-8B-Instruct",
           transformer_type=transformers.AutoModelForCausalLM,
           access_token=HUGGINGFACE_TOKEN,
       ),
       # it is assumed for text related tasks, you have 'text' column in the dataset.
       # for more info on how dataset is loaded check load_and_preprocess_data function in sdk/python/kubeflow/trainer/hf_llm_training.py
       dataset_provider_parameters=HuggingFaceDatasetParams(repo_id="aishwaryayyy/events_data"),
       trainer_parameters=HuggingFaceTrainerParams(
           lora_config=LoraConfig(
                r=16,
               lora_alpha=32,
               lora_dropout=0.1,
               bias="none",
               task_type="CAUSAL_LM",
               target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
           ),
           training_parameters=TrainingArguments(
               max_grad_norm=0.4,
               num_train_epochs=3,
               per_device_train_batch_size=4,
               gradient_accumulation_steps=8,
               gradient_checkpointing=True,
               gradient_checkpointing_kwargs={
                   "use_reentrant": False
               },  # this is mandatory if checkpointng is enabled
               warmup_steps=8,
               learning_rate=2e-4,
               lr_scheduler_type="cosine",
               bf16=True,
               logging_steps=0.01,
               output_dir=OUTPUT,
               optim=f"paged_adamw_32bit",
               save_steps=0.01,
               save_total_limit=3,
               disable_tqdm=False,
               resume_from_checkpoint=True,
               remove_unused_columns=True,
               # ddp_backend="gloo",  # change the backend to gloo if you want cpu based training and remove the gpu key in resources_per_worker
           ),
       ),
       resources_per_worker={
           "gpu": 1,
           "cpu": 28,
           "memory": "60Gi",
       },  # remove the gpu key if you don't want to attach gpus to the pods
    )
    
    # check the status of the job
    import time

    time.sleep(30)

    while True:
        conditions = client.get_job_conditions(name='llama-3-1-8b-kubecon', job_kind=constants.PYTORCHJOB_KIND)
        for condition in conditions:
            if condition.type == 'Succeeded' and condition.status == 'True':
                print("pytorch job has succeeded :)")
            elif condition.type in ['Failed', 'Terminated'] and condition.status == 'True':
                print("pytorch job has failed :(")
        print("waiting for pytorch job to finish")
        time.sleep(10)

Merge (Parameter Efficient Fine-Tuning) PEFT model weights with the pretrained model to form the fine-tuned model.
 
Store it on a Persistent Volume shared across Kubeflow Pipeline tasks.
Also, save the tokenizer along with the fine-tuned model.

In [ ]:
@dsl.component(base_image='quay.io/aishquaya/kfp-python:latest')
def store_model():
    from transformers import AutoTokenizer, AutoModelForCausalLM
    import torch
    from peft import PeftModelForCausalLM

    HUGGINGFACE_TOKEN = "YOUR_HUGGINGFACE_TOKEN"
    
    model = AutoModelForCausalLM.from_pretrained(
       "meta-llama/Llama-3.1-8B-Instruct",
       torch_dtype=torch.float16,
       low_cpu_mem_usage=True,
       device_map="cuda:0",
        token=HUGGINGFACE_TOKEN,
    )
    
    model = PeftModelForCausalLM.from_pretrained(
       model, "storage-initializer/output/llama-3.1-8B-kubecon/checkpoint-12"
    )

    finetuned_model = model.merge_and_unload()
    finetuned_model.save_pretrained("storage-initializer/serve_model/llama-3.1-8B-kubecon")

    pretrained_model = "meta-llama/Llama-3.1-8B-Instruct"
    tokenizer = AutoTokenizer.from_pretrained(pretrained_model, token=HUGGINGFACE_TOKEN)
    tokenizer.save_pretrained("storage-initializer/serve_model/llama-3.1-8B-kubecon")

## Serve the Fine-Tuned Model

This component serves the fine-tuned model using Kserve. Create an InferenceService with HuggingFace runtime and *[6 vCPUs, 24Gi Memory and 1 GPU]* resource configuration. 

Specify the fine-tuned model's location in the storage_uri field.

In [ ]:
@dsl.component(packages_to_install=['kserve', 'git+https://github.com/kubernetes-client/python.git'])
def serve_model():
    from kubernetes import client 
    from kserve import KServeClient
    from kserve import constants
    from kserve import utils
    from kserve import V1beta1InferenceService
    from kserve import V1beta1InferenceServiceSpec
    from kserve import V1beta1PredictorSpec
    from kserve import V1beta1ModelSpec
    from kserve import V1beta1ModelFormat
    import kubernetes.client
    from kubernetes.client import V1ResourceRequirements

    namespace = utils.get_default_target_namespace()

    api_version = constants.KSERVE_GROUP + '/' + "v1beta1"
    
    isvc = V1beta1InferenceService(
        api_version=api_version,
    	kind="InferenceService",
        metadata=client.V1ObjectMeta(name='llama-3-1-8b-kubecon', namespace=namespace),
        spec=V1beta1InferenceServiceSpec(
            predictor=V1beta1PredictorSpec(
                model=V1beta1ModelSpec(
                model_format=V1beta1ModelFormat(name='huggingface'),
                image='kserve/huggingfaceserver:latest',
                storage_uri='pvc://storage-initializer/serve_model/llama-3.1-8B-kubecon',
                resources=V1ResourceRequirements(
                    limits={'cpu': '6','memory': '24Gi', 'nvidia.com/gpu': '1'},
                    requests={'cpu': '6','memory': '24Gi', 'nvidia.com/gpu': '1'}
                )
            )
    )))

    KServe = KServeClient()
    KServe.create(isvc)

Initialize the Pipeline and link all the above declared tasks specifying their dependencies with each other.

We have mounted a Persistent Volume Claim (PVC) to share storage space across Kubeflow components. The fine-tuned model will be stored here.

In [ ]:
@dsl.pipeline(name='finetune-pipeline')
def e2e_ml_pipeline():
    from kfp import kubernetes
    provision_model_storage = kubernetes.CreatePVC(
        # can also use pvc_name instead of pvc_name_suffix to use a pre-existing PVC
        pvc_name='storage-initializer',
        access_modes=['ReadWriteOnce'],
        size='100Gi',
        storage_class_name='nfs-storageclass',
    )

    training_task = finetune_model()
    merging_task = store_model()
    serving_task = serve_model()
    training_task.after(provision_model_storage)
    merging_task.after(training_task)
    serving_task.after(merging_task)

    serving_task.set_caching_options(False)
    merging_task.set_caching_options(False)
    # training_task.set_caching_options(False)
    
    merging_task.add_node_selector_constraint('nvidia.com/gpu')
    merging_task.set_gpu_limit(1)
    
    kubernetes.mount_pvc(
        merging_task,
        pvc_name=provision_model_storage.outputs['name'],
        mount_path='/storage-initializer',
    )

Create a run for the pipeline using Kubeflow Pipeline Client.

In [ ]:
kfp_client = client.Client()
run = kfp_client.create_run_from_pipeline_func(
    e2e_ml_pipeline,
    arguments={},
)

Once all the Kubeflow tasks in the pipeline are completed, the fine-tuned model should be ready for inference requests. You can port-forward the Inference pod and execute inference requests as shown below.

In [ ]:
# ! curl --location 'http://localhost:8083/openai/v1/chat/completions' \
# --header 'Content-Type: application/json' \
# --data '{ "model": "llama-3-1-8b-kubecon", "messages": [{ "role": "user", "content": "Can you tell me when is KubeCon + CloudNativeCon India 2024 scheduled?"}], "max_tokens": 200, "stream": false}' | grep -o '"content":"[^"]*"' \
# | sed 's/"content":"\(.*\)"/\1/'